In [1]:
#model套件
import torch
from torch import nn as nn
#資料處理套件
import numpy as np
#時間套件
import time
#列舉套件
from enum import Enum

import sys
#將父目錄添加到模組搜尋路徑中
sys.path.append('..')
#取得模型
import GetModel as GM
#訓練/測試 模型
import TrainModel as TM
#資料前處理
import TrainDataPreprocessing as TDP

In [2]:
#存檔位置
ModelMode = "ESC10_kFold"
TrainDataSoundMode = ["_1s", "_3s", "_5s", "_135s", "_531s"]
sMode = 2
filePath_Train = ModelMode + TrainDataSoundMode[sMode] + "_Train.txt"
filePath_LastData = ModelMode + TrainDataSoundMode[sMode] +  "_LastData.txt"
modelPathHead = ModelMode + TrainDataSoundMode[sMode]

In [3]:
#紀錄開始時間
dataload_start_time = time.time()

#資料讀取
print("讀取1秒資料")
data_1S = TDP.loadDataESC10(soundLength=1)
print("讀取3秒資料")
data_3S = TDP.loadDataESC10(soundLength=3)
print("讀取5秒資料")
data_5S = TDP.loadDataESC10(soundLength=5)

#紀錄結束時間
dataload_end_time = time.time()

#計算耗時
dataload_training_time = dataload_end_time - dataload_start_time

#印出耗時
print("Data Loading Time: {:.2f} seconds".format(dataload_training_time))

讀取1秒資料
讀取3秒資料
讀取5秒資料
Data Loading Time: 0.19 seconds


In [4]:
kFoldTrainTime = []
kFoldTrainAccuracy = []
kFoldTestAccuracy1 = []
kFoldTestAccuracy3 = []
kFoldTestAccuracy5 = []


#設定存檔位置
f = open(filePath_Train, "w")
#k-fold訓練
for fold in range(1,6):
    print("----------------------------------------------------------------------------------------------------------------")
    print("fold: ", fold)
    print("----------------------------------------------------------------------------------------------------------------")

    #當前裝置能使用的GPU數量
    num_devices = torch.cuda.device_count()
    print("可用的 GPU 数量:", num_devices)

    #建立模型
    model = GM.getModel(num_classes=10)

    #將模型放置到 GPU 
    model = model.cuda()

    #使用 nn.DataParallel 包装模型
    #進行併行處理
    model = nn.DataParallel(model)
    #------------------------------------------------
    #需要調整的地方
    #------------------------------------------------
    #訓練資料(要用幾秒訓練) data_1S/data_3S/data_5S
    trainData = data_5S
    #------------------------------------------------
    
    #紀錄訓練開始時間
    start_time = time.time()

    #訓練模型
    model, train_loss, train_accuracy = TM.train_model_TrainAll(model=model,
                                                                inputData=trainData,
                                                                trainFold=fold)
    
    #測試準確度
    test_accuracy1 = TM.test_model(model=model, inputData=data_1S, testFold=fold)
    test_accuracy3 = TM.test_model(model=model, inputData=data_3S, testFold=fold)
    test_accuracy5 = TM.test_model(model=model, inputData=data_5S, testFold=fold)

    #紀錄訓練結束時間
    end_time = time.time()

    #計算訓練耗時
    training_time = end_time - start_time

    #印出訓練耗時
    print("Training Time: {:.2f} seconds".format(training_time))

    f.write("---------------------------------------------------------------------------------------------------------------\n")
    f.write("訓練過程\n")
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    for i in range(len(train_loss)):
        # 寫入資料
        f.write("Fold: {} Train Epoch: {}/{} Train_Loss: {} Train_Acc: {:.6f}%\n".format(fold, i+1, len(train_loss), train_loss[i], train_accuracy[i]))
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    f.write("Test_Accuracy1: {:.6f}% Test_Accuracy3: {:.6f}% Test_Accuracy5: {:.6f}%\n".format(test_accuracy1, test_accuracy3, test_accuracy5))
    f.write("Training Time: {:.2f} seconds\n".format(training_time))
    f.write("---------------------------------------------------------------------------------------------------------------\n")

    #--------------------------------------------------------------------

    
    kFoldTrainTime.append(training_time)
    kFoldTrainAccuracy.append(train_accuracy[-1])
    kFoldTestAccuracy1.append(test_accuracy1)
    kFoldTestAccuracy3.append(test_accuracy3)
    kFoldTestAccuracy5.append(test_accuracy5)
    
    modelpath = modelPathHead + "_" + str(fold) + "fold.pth"
    #保存模型
    torch.save(model.state_dict(), modelpath)
    
#關閉
f.close()

----------------------------------------------------------------------------------------------------------------
fold:  1
----------------------------------------------------------------------------------------------------------------
可用的 GPU 数量: 1


C:\Users\test\anaconda3\envs\pytorch\lib\site-packages\torchvision\ops\misc.py:120: UserWarning: Don't use ConvNormActivation directly, please use Conv2dNormActivation and Conv3dNormActivation instead.
  warnings.warn(


Number of classes defined: 10, but try to load pre-trained layer with logits: 527
Dropping last layer.
Error(s) in loading state_dict for MobileNetV3:
	Missing key(s) in state_dict: "classifier.5.weight", "classifier.5.bias". 
Loading weights pre-trained weights in a non-strict manner.
MobileNetV3(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
          (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): ConvNormActivation(
          (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=Fa

C:\Users\test\anaconda3\envs\pytorch\lib\site-packages\torch\functional.py:641: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\SpectralOps.cpp:867.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]


Train Epoch: 1/50 Train_Loss: 2.359160900115967 Train_acc: 8.750000%
Train Epoch: 2/50 Train_Loss: 2.2871503829956055 Train_acc: 8.750000%
Train Epoch: 3/50 Train_Loss: 2.293900966644287 Train_acc: 9.687500%
Train Epoch: 4/50 Train_Loss: 2.300558090209961 Train_acc: 11.562500%
Train Epoch: 5/50 Train_Loss: 2.294428586959839 Train_acc: 13.125000%
Train Epoch: 6/50 Train_Loss: 2.2207915782928467 Train_acc: 15.312500%
Train Epoch: 7/50 Train_Loss: 2.2224111557006836 Train_acc: 20.625000%
Train Epoch: 8/50 Train_Loss: 2.1365742683410645 Train_acc: 24.062500%
Train Epoch: 9/50 Train_Loss: 2.0863571166992188 Train_acc: 33.750000%
Train Epoch: 10/50 Train_Loss: 1.9230059385299683 Train_acc: 52.812500%
Train Epoch: 11/50 Train_Loss: 1.8491228818893433 Train_acc: 72.187500%
Train Epoch: 12/50 Train_Loss: 1.684927225112915 Train_acc: 81.562500%
Train Epoch: 13/50 Train_Loss: 1.5768903493881226 Train_acc: 90.000000%
Train Epoch: 14/50 Train_Loss: 1.4969347715377808 Train_acc: 92.812500%
Train Epo

Train Epoch: 1/50 Train_Loss: 2.3638603687286377 Train_acc: 8.750000%
Train Epoch: 2/50 Train_Loss: 2.3608415126800537 Train_acc: 9.062500%
Train Epoch: 3/50 Train_Loss: 2.380664825439453 Train_acc: 8.750000%
Train Epoch: 4/50 Train_Loss: 2.3004062175750732 Train_acc: 8.437500%
Train Epoch: 5/50 Train_Loss: 2.3009982109069824 Train_acc: 10.000000%
Train Epoch: 6/50 Train_Loss: 2.287003755569458 Train_acc: 11.562500%
Train Epoch: 7/50 Train_Loss: 2.276682138442993 Train_acc: 17.812500%
Train Epoch: 8/50 Train_Loss: 2.2120985984802246 Train_acc: 22.812500%
Train Epoch: 9/50 Train_Loss: 2.0890986919403076 Train_acc: 35.937500%
Train Epoch: 10/50 Train_Loss: 2.002768039703369 Train_acc: 56.875000%
Train Epoch: 11/50 Train_Loss: 1.8841221332550049 Train_acc: 64.062500%
Train Epoch: 12/50 Train_Loss: 1.7610321044921875 Train_acc: 76.250000%
Train Epoch: 13/50 Train_Loss: 1.6464900970458984 Train_acc: 83.125000%
Train Epoch: 14/50 Train_Loss: 1.525773286819458 Train_acc: 87.812500%
Train Epoc

Train Epoch: 1/50 Train_Loss: 2.2703959941864014 Train_acc: 13.125000%
Train Epoch: 2/50 Train_Loss: 2.2987170219421387 Train_acc: 10.625000%
Train Epoch: 3/50 Train_Loss: 2.2766952514648438 Train_acc: 13.437500%
Train Epoch: 4/50 Train_Loss: 2.2963991165161133 Train_acc: 12.812500%
Train Epoch: 5/50 Train_Loss: 2.2325334548950195 Train_acc: 16.875000%
Train Epoch: 6/50 Train_Loss: 2.266719102859497 Train_acc: 17.812500%
Train Epoch: 7/50 Train_Loss: 2.188338041305542 Train_acc: 23.437500%
Train Epoch: 8/50 Train_Loss: 2.1030678749084473 Train_acc: 34.687500%
Train Epoch: 9/50 Train_Loss: 1.9717720746994019 Train_acc: 45.000000%
Train Epoch: 10/50 Train_Loss: 1.9357194900512695 Train_acc: 60.625000%
Train Epoch: 11/50 Train_Loss: 1.7729249000549316 Train_acc: 77.187500%
Train Epoch: 12/50 Train_Loss: 1.6878682374954224 Train_acc: 81.562500%
Train Epoch: 13/50 Train_Loss: 1.5497567653656006 Train_acc: 85.000000%
Train Epoch: 14/50 Train_Loss: 1.397256851196289 Train_acc: 92.812500%
Trai

Train Epoch: 1/50 Train_Loss: 2.3426473140716553 Train_acc: 9.375000%
Train Epoch: 2/50 Train_Loss: 2.344609260559082 Train_acc: 9.375000%
Train Epoch: 3/50 Train_Loss: 2.359306573867798 Train_acc: 9.062500%
Train Epoch: 4/50 Train_Loss: 2.3115978240966797 Train_acc: 11.875000%
Train Epoch: 5/50 Train_Loss: 2.2823076248168945 Train_acc: 11.250000%
Train Epoch: 6/50 Train_Loss: 2.2556159496307373 Train_acc: 13.437500%
Train Epoch: 7/50 Train_Loss: 2.2128374576568604 Train_acc: 15.625000%
Train Epoch: 8/50 Train_Loss: 2.1177327632904053 Train_acc: 27.812500%
Train Epoch: 9/50 Train_Loss: 2.0782947540283203 Train_acc: 36.875000%
Train Epoch: 10/50 Train_Loss: 1.9303560256958008 Train_acc: 52.812500%
Train Epoch: 11/50 Train_Loss: 1.8015402555465698 Train_acc: 70.937500%
Train Epoch: 12/50 Train_Loss: 1.6990076303482056 Train_acc: 82.187500%
Train Epoch: 13/50 Train_Loss: 1.624021053314209 Train_acc: 86.250000%
Train Epoch: 14/50 Train_Loss: 1.4917917251586914 Train_acc: 92.812500%
Train E

Train Epoch: 1/50 Train_Loss: 2.3248472213745117 Train_acc: 14.062500%
Train Epoch: 2/50 Train_Loss: 2.348097085952759 Train_acc: 15.312500%
Train Epoch: 3/50 Train_Loss: 2.3594915866851807 Train_acc: 13.750000%
Train Epoch: 4/50 Train_Loss: 2.2804696559906006 Train_acc: 16.875000%
Train Epoch: 5/50 Train_Loss: 2.3402678966522217 Train_acc: 17.187500%
Train Epoch: 6/50 Train_Loss: 2.238461971282959 Train_acc: 18.125000%
Train Epoch: 7/50 Train_Loss: 2.232071876525879 Train_acc: 25.000000%
Train Epoch: 8/50 Train_Loss: 2.1223561763763428 Train_acc: 31.250000%
Train Epoch: 9/50 Train_Loss: 2.0735526084899902 Train_acc: 41.562500%
Train Epoch: 10/50 Train_Loss: 1.9285989999771118 Train_acc: 52.187500%
Train Epoch: 11/50 Train_Loss: 1.8329355716705322 Train_acc: 65.000000%
Train Epoch: 12/50 Train_Loss: 1.6628233194351196 Train_acc: 74.375000%
Train Epoch: 13/50 Train_Loss: 1.6099895238876343 Train_acc: 82.187500%
Train Epoch: 14/50 Train_Loss: 1.502995252609253 Train_acc: 90.937500%
Train

In [5]:
#設定存檔位置
f = open(filePath_LastData, "w")

#整理出訓練完的數值
f.write("---------------------------------------------------------------------------------------------------------------\n")
f.write("訓練結果\n")
f.write("-----------------------------------------------------------------------------------\n")
for fold in range(5):
    # 寫入資料
    f.write("Fold: {} Train_acc: {:.10f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}% Training Time: {:.2f} seconds\n".format(fold+1, kFoldTrainAccuracy[fold], kFoldTestAccuracy1[fold], kFoldTestAccuracy3[fold], kFoldTestAccuracy5[fold], kFoldTrainTime[fold]))
    f.write("-----------------------------------------------------------------------------------\n")
# 寫入資料
f.write("Max Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(max(kFoldTrainAccuracy), max(kFoldTestAccuracy1), max(kFoldTestAccuracy3), max(kFoldTestAccuracy5)))
f.write("Min Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(min(kFoldTrainAccuracy), min(kFoldTestAccuracy1), min(kFoldTestAccuracy3), min(kFoldTestAccuracy5)))
f.write("Avg Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(np.mean(kFoldTrainAccuracy), np.mean(kFoldTestAccuracy1), np.mean(kFoldTestAccuracy3), np.mean(kFoldTestAccuracy5)))

f.close()